In [ ]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as go
#! pip install kaleido
import kaleido

In [2]:
#Planet orbit animations:

directory = '/Users/gillislowry/Documents/Kaltenegger/'
filename = 'ExoplanetVisualizationFullPrototype.csv'
PlanetSystems = pd.read_csv(directory + filename)



# Columns needed: 'HostStarName','StarMass','Temp', then 'Name','Size','Orb','Ecc','x','y' for each planet
df = pd.DataFrame(columns = ['HostStarName','Name0','Name1','Name2','Name3','Name4','Name5','Name6','Name7'])

for i, row in PlanetSystems.iterrows(): 
    
    df.at[i, 'HostStarName'] = str(row['HostStarName'])
    if row['Temp'] != 0 and row['Temp'] != ' ' and row['Temp'] != '':
        df.at[i, 'Temp'] = float(row['Temp'])
    else:
        if row['StarColor'] != ' ':
            if 'orange' in row['StarColor']:
                df.at[i, 'Temp'] = 4000
            elif 'red' in row['StarColor']:
                df.at[i, 'Temp'] = 3300
            elif 'yellow' in row['StarColor']:
                df.at[i, 'Temp'] = 5700
            elif 'white' in row['StarColor']:
                df.at[i, 'Temp'] = 6138
            elif 'blue' in row['StarColor']:
                df.at[i, 'Temp'] = 7000
            elif 'brown' in row['StarColor']:
                df.at[i, 'Temp'] = 600
            else:
                df.at[i, 'Temp'] = 5000
        else:
            df.at[i, 'Temp'] = 5000
    if row['StarMass'] != 0:
            df.at[i, 'StarMass'] = 1.989e30*row['StarMass']  # Mass of the star (in kg)
    else:
            df.at[i, 'StarMass'] = 1.989e30
            
    for k in range(0,8): #fill dataframe with planet info
        
        if row['Name'+str(k)] != ' ' and row['Name'+str(k)] != 0:
            df.at[i, 'Name'+str(k)] = str(row['Name'+str(k)])
            if row['Radius'+str(k)] != 0 and row['Radius'+str(k)] != ' ':
                df.at[i, 'Size'+str(k)] = row['Radius'+str(k)]
            elif row['Mass'+str(k)] != 0 and row['Mass'+str(k)] != ' ':
                if row['Mass'+str(k)] > 5:
                    df.at[i, 'Size'+str(k)] = (4+np.emath.logn(5, row['Mass'+str(k)]))/(2.5)
                else:
                    df.at[i, 'Size'+str(k)] = row['Mass'+str(k)]/(2.5)
            else:
                df.at[i, 'Size'+str(k)] = 1
            df.at[i, 'Orb'+str(k)] = row['OrbDist'+str(k)]
            df.at[i, 'Orb_in_meters'+str(k)] = (row['OrbDist'+str(k)])*(149600000000)
            df.at[i, 'OrbPer'+str(k)] = row['OrbPer'+str(k)]      
            if row['Ecc'+str(k)] != ' ':
                df.at[i, 'Ecc'+str(k)] = row['Ecc'+str(k)]
            else:
                df.at[i, 'Ecc'+str(k)] = 0
       
            
    df.at[i, 'NumberStars'] = row['NumberStars']
    df.at[i, 'RecentVenus'] = row['RecentVenus']
    df.at[i, 'RunawayGreenhouse'] = row['RunawayGreenhouse']
    #df.at[i, 'MoistGreenhouse'] = row['MoistGreenhouse']
    df.at[i, 'MaximumGreenhouse'] = row['MaximumGreenhouse']
    df.at[i, 'EarlyMars'] = row['EarlyMars']
                  
df = df.fillna(0)        
print(df)
print(df['OrbPer0'])
df.to_csv('VisalizationOrbit.csv', index=True)

In [123]:
from plotly.subplots import make_subplots

directory = '/Users/gillislowry/Documents/Kaltenegger/'
filename = 'VisalizationOrbit.csv'
df = pd.read_csv(directory + filename)

def graph(df):
    
    for i, row in df.iterrows(): 

        
        print(row['HostStarName'])

        # Constants
        G = 6.67430e-11  # Gravitational constant  

        #Create gradient for star color based on stellar temp
        scale = [(0.0,"#522020"),(0.07,"#800000"),(0.09,"#b80000"),(0.13,"#ede968"),(0.155,"#e8e8e8"),(0.3,"#97dadb"),(1.0,"#757aff")]
        backgroundcolor = "rgb(0,2,38)"

        T = {}
        T_list = []
        orbs = []
        orbs_au = []
        eccs = []
        color = float(row['Temp']) 

        # Find maximum time to complete an orbit (this will be half of the total animation runtime)
        for k in range(0,8):
            
            if row['Orb_in_meters'+str(k)] != 0:
                print(row['Orb_in_meters'+str(k)])
            if row['OrbPer'+str(k)] != 0:
                print(row['OrbPer'+str(k)])
            if row['Name'+str(k)] != 0:
                print(row['Name'+str(k)])

            if row['Orb'+str(k)] != 0.0 and row['Name'+str(k)] != 0: 
                T[row['Name'+str(k)]] = ( 2 * np.pi * np.sqrt(row['Orb_in_meters'+str(k)]**3 / (G * row['StarMass'])))
                T_list.append( 2 * np.pi * np.sqrt(row['Orb_in_meters'+str(k)]**3 / (G * row['StarMass'])))
                orbs.append(row['Orb_in_meters'+str(k)])
                orbs_au.append(row['Orb'+str(k)])
                eccs.append(row['Ecc'+str(k)])
                
                    
            elif row['Orb'+str(k)] == 0.0 and row['Name'+str(k)] != 0 and row['OrbPer'+str(k)] != 0:
                T[row['Name'+str(k)]] = row['OrbPer'+str(k)]*24*60*60
                T_list.append(row['OrbPer'+str(k)]*24*60*60)
                orbs.append(((G * row['StarMass']*(row['OrbPer'+str(k)]*24*60*60)**2) / (4*np.pi**2))**(1/3))
                orbs_au.append((((G * row['StarMass']*(row['OrbPer'+str(k)]*24*60*60)**2) / (4*np.pi**2))**(1/3))/1.496e11)
                eccs.append(row['Ecc'+str(k)])
                    
            elif row['Name'+str(k)] != 0:
                print("bad")
                T[row['Name'+str(k)]] = 0
                T_list.append(0)
                orbs.append(np.nan)
                orbs_au.append(np.nan)
                eccs.append(np.nan)
            
            if len(eccs) !=0 and np.max(eccs) > 0.5:
                ecc_adjust = 1.2
                ecc_adjust2 = 1.3
                if np.max(eccs) > 0.8:
                    ecc_adjust = 1.4
                    ecc_adjust2 = 1.6
            else:
                ecc_adjust2 = 0
                ecc_adjust = 1

        #print(T_list)    
                  
        if np.max(T_list) == 0:
            noData = True
        else:
            noData = False
            #print(str(row['Name'+str(k)]) + " " + str(T[row['Name'+str(k)]]))


        # Create time array
        if noData != True:
            if np.min(orbs_au) > 2 and np.min(orbs_au) <= 100:
                frames = 612
                t = np.linspace(0, np.max(T_list), frames)
                annotation_divide = 612
            elif np.min(orbs_au) > 100:
                frames = 1200
                t = np.linspace(0, np.max(T_list), frames)
                annotation_divide = 1200
            else:
                frames = 612
                t = np.linspace(0, 2*np.max(T_list), frames)
                annotation_divide = 306
        else:
            frames = 1
            t = np.linspace(0, 1, frames)
            annotation_divide = 1

        # Determine whether time annotation should be in hours, days, or years:
        if noData != True and np.max(T_list) <= 100000:
            unit = " hours"
        elif noData != True and np.max(T_list) <= 50000000:
            unit = " days"
        elif noData != True:
            unit = " years"
        else:
            unit = ''

        # Create empty frame list:
        frames_data2 = []

        for j in range(0, len(t)): # For each frame,

                #print("Frame " + str(j))
                
                planet2 = []

                #Determine time to be annotated on each frame:
                if noData != True and np.max(T_list) <= 100000:
                    time = np.round(((np.max(T_list)/annotation_divide)*j)/60/60,3)
                elif noData != True and np.max(T_list) <= 50000000: 
                    time = np.round(((np.max(T_list)/annotation_divide)*j)/60/60/24,3)
                elif noData != True:
                    time = np.round(((np.max(T_list)/annotation_divide)*j)/60/60/24/365.256363004,3)
                else:
                    time = ''

                for k in range(0, len(T)): # For each planet,

                    if row['Name'+str(k)] != 0 and noData != True:

                        #theta = 2 * np.pi * t / T[row['Name'+str(k)]]
                        #r = row['Orb_in_meters'+str(k)] * (1 - row['Ecc'+str(k)] ** 2) / (1 + row['Ecc'+str(k)] * np.cos(2 * np.pi * t / T[row['Name'+str(k)]]))
                        # x = (r * np.cos(theta))
                        # y = (r * np.sin(theta))
                        x = list((row['Orb_in_meters'+str(k)] * (1 - float(row['Ecc'+str(k)]) ** 2) / (1 + float(row['Ecc'+str(k)]) * np.cos(2 * np.pi * t / T[row['Name'+str(k)]])) * np.cos(2 * np.pi * t / T[row['Name'+str(k)]]))+2*(row['Orb_in_meters'+str(k)]*float(row['Ecc'+str(k)])))
                        y = list(row['Orb_in_meters'+str(k)] * (1 - float(row['Ecc'+str(k)]) ** 2) / (1 + float(row['Ecc'+str(k)]) * np.cos(2 * np.pi * t / T[row['Name'+str(k)]])) * np.sin(2 * np.pi * t / T[row['Name'+str(k)]]))
                        # Non-eccentricity version:
                        # x = list(row['Orb_in_meters'+str(k)] * np.cos(2 * np.pi * t / T[row['Name'+str(k)]]))
                        # y = list(row['Orb_in_meters'+str(k)] * np.sin(2 * np.pi * t / T[row['Name'+str(k)]]))
                        orbdist = orbs_au[k]

                        data_by_planet = {
                        "x": [x[j]],
                        "y": [y[j]],
                        "mode": "markers+text",
                        "marker": {
                            "size": (5 + row['Size'+str(k)]),
                            "color": "rgb(236, 85, 250)"
                        },
                        "name": (row['HostStarName']+ ' ' + row['Name'+str(k)]),
                        "customdata": [orbdist],
                        "hovertemplate": "<i>Orbital distance</i>: %{customdata} AU",
                        "textposition": 'top center'
                        }

                        graph_one_planet = go.Scatter(data_by_planet)
                        graph_one_orbit = go.Scatter(
                            x=x, y=y, mode='markers', hoverinfo='skip', marker=dict(size=1, color='white')
                            )

                        # Add each planet's position to planet list
                        planet2.append(graph_one_planet)
                        planet2.append(graph_one_orbit)

                if noData != True and np.max(T_list) > 0.0:
                    frame_layout = go.Layout(annotations=
                                    [go.layout.Annotation(text="Time elapsed: " + str(time) + unit,
                                      xref="paper", yref="paper",
                                        font = dict(color = 'white'),
                                      x=0.92, y=0.9, showarrow=False)],
                                    )      
                else:
                    frame_layout = go.Layout(annotations=
                                [go.layout.Annotation(text="No orbit data for " + str(row['HostStarName']),
                              xref="paper", yref="paper",
                                font = dict(color = 'white', size = 15),
                              x=0.5, y=0.55, showarrow=False)],)


                # Graph star for each frame:
                graph_star = go.Scatter(
                        x=[0,0,0], y=[0,0,0], mode='markers', marker=dict(color = [400,40000,color], coloraxis='coloraxis', size=10),
                        name=row['HostStarName'],
                        hoverinfo='skip'
                        )

                planet2.append(graph_star)

                # On second image, graph Earth orbit
                T_earth = ( 2 * np.pi * np.sqrt(1.496e11**3 / (G * 1.989e30)))
                x_earth = list((1.496e11 * (1 - 0.017 ** 2) / (1 + 0.017 * np.cos(2 * np.pi * t / T_earth)) * np.cos(2 * np.pi * t / T_earth))+2*(1.496e11*0.017))
                y_earth = list(1.496e11 * (1 - 0.017 ** 2) / (1 + 0.017 * np.cos(2 * np.pi * t / T_earth)) * np.sin(2 * np.pi * t / T_earth))
                x_earth_orb = list((1.496e11 * (1 - 0.017 ** 2) / (1 + 0.017 * np.cos(2 * np.pi * 15*t / T_earth)) * np.cos(2 * np.pi * 15*t / T_earth))+2*(1.496e11*0.017))
                y_earth_orb = list(1.496e11 * (1 - 0.017 ** 2) / (1 + 0.017 * np.cos(2 * np.pi * 15*t / T_earth)) * np.sin(2 * np.pi * 15*t / T_earth))

                graph_earth_orbit = go.Scatter(
                        x=x_earth_orb, y=y_earth_orb, mode='markers', hoverinfo='skip', marker=dict(size=1, color='rgb(57, 194, 200)')
                        )
                planet2.append(graph_earth_orbit)

                data_earth = {
                    "x": [x_earth[j]],
                    "y": [y_earth[j]],
                    "mode": "markers+text",
                    "marker": {
                        "size": (6),
                        "color": "cyan"
                    },
                    "name": ("Earth"),
                    "customdata": ['1'],
                    "hovertemplate": "<i>Orbital distance</i>: %{customdata} AU",
                    "textposition": 'top center'
                    }

                graph_earth = go.Scatter(data_earth)
                planet2.append(graph_earth)


                # Add list of planet positions to the frame for each image
                frame2 = (go.Frame(data=planet2,layout=frame_layout))

                # Append one frame 
                frames_data2.append(frame2)


        # Create plot
        fig2 = go.Figure(data=[go.Scatter()
                              for traces in planet2])

        # Set frames attribute
        fig2.frames = frames_data2
        for k in range(len(fig.frames)):
            fig2.frames[k]['layout'].update()


        # Update layout
        
        if noData != True: 
            range_x = [-1.551 * np.max(orbs)*ecc_adjust+ecc_adjust2*np.max(orbs), 1.551 * np.max(orbs)*ecc_adjust+ecc_adjust2*np.max(orbs)]
            range_y = [-1.11 * np.max(orbs)*ecc_adjust, 1.11 * np.max(orbs)*ecc_adjust]
        else:
            range_x = [-1,1]
            range_y = [-1,1]
            
        if noData != True: 
            minrange_x = [-1.551 * np.min(orbs)*ecc_adjust, 1.551 * np.min(orbs)*ecc_adjust]
            minrange_y = [-1.11 * np.min(orbs)*ecc_adjust, 1.11 * np.min(orbs)*ecc_adjust]
        else:
            minrange_x = [-1,1]
            minrange_y = [-1,1]
        

        fig2.update_layout(
            xaxis=dict(
                range=range_x,
                showgrid=False,
                zeroline=False,
                showline=False,
                showticklabels=False
            ),
            yaxis=dict(
                range=range_y,
                showgrid=False,
                zeroline=False,
                showline=False,
                showticklabels=False
            ),
            coloraxis_showscale=False,
            coloraxis=dict(colorscale = scale),
            plot_bgcolor=backgroundcolor, 
            paper_bgcolor=backgroundcolor,
            showlegend=False,
            margin = dict(l = 0, r = 0, b = 0, t = 0),
            width=1400, 
            height=1000, 
            updatemenus=[{
                'buttons': [
                                    {
                        'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                        'label': 'Pause/Stop',
                        'method': 'animate'
                    },
                    {
                        'args': [None, {'frame': {'duration': 0, 'redraw': False}, 'transition': {'duration': 800}, 'fromcurrent': True}],
                        'label': 'Play faster',
                        'method': 'animate'
                    },
                    {
                        'args': [None, {'frame': {'duration': 0, 'redraw': True}, 'transition': {'duration': 800}, 'fromcurrent': True}],
                        'label': 'Play with time counter',
                        'method': 'animate'
                    },
                    {
                        'args': [{'xaxis.range':[-1.551 * 1.496e+11, 1.551 *1.496e+11],
                                 'yaxis.range':[-1.11 *1.496e+11, 1.11 *1.496e+11]}],
                        'label': 'Compare with Earth\'s orbit',
                        'method': 'relayout'
                    },
                    {
                        'args': [{'xaxis.range':minrange_x,
                                 'yaxis.range':minrange_y}],
                        'label': 'Zoom to closest orbit',
                        'method': 'relayout'
                    },
                    {
                        'args': [{'xaxis.range':range_x,
                                 'yaxis.range':range_y}],
                        'label': 'Reset Zoom',
                        'method': 'relayout'
                    }
                ],
                'direction': 'left',
                'showactive': True,
                'type': 'buttons',
                'x': 0.5,
                'xanchor': 'center',
                'y': 1.0,
                'yanchor': 'top',
                'bgcolor': 'white', 
                'font': {'color':'black', 'family':'avenir'}
            }]
            )

        print(row['HostStarName'] + " figure 2 done!")
        fig2.show()
        if len(T_list) <= 2 or noData == True: 
            showtextOnAutoplay = True
        else:
            showtextOnAutoplay = False

        fig2.write_html("/Users/gillislowry/Documents/Kaltenegger/orbit animations/"+row['HostStarName']+"_earth.html", 
                       auto_play = True, animation_opts = dict(frame = dict(duration = 0, redraw = showtextOnAutoplay), 
                        transition = dict(duration = 800), fromcurrent = True),
                      config = {'scrollZoom': True, 'displayModeBar': True, 
                                'modeBarButtonsToRemove': ['autoScale2d', 'select2d', 'lasso2d']})


In [ ]:
graph(df)  # process 
        
